In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

print("TensorFlow:", tf.__version__)


TensorFlow: 2.20.0


In [2]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

DATA_DIR = "./"
CSV_PATH = os.path.join(DATA_DIR, "train.csv")
IMAGES_DIR = os.path.join(DATA_DIR, "images")

N_MALIGNANT = 4521
N_BENIGN = 4521

TRAIN_SIZE = 0.80
VAL_SIZE = 0.10
TEST_SIZE = 0.10

assert abs(TRAIN_SIZE + VAL_SIZE + TEST_SIZE - 1.0) < 1e-6

IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 40

OUT_DIR = "./modelos_guardados_isic2019"
os.makedirs(OUT_DIR, exist_ok=True)
BEST_MODEL_PATH = os.path.join(OUT_DIR, "isic2019_mobilenetv2_best.keras")

print("CSV_PATH:", CSV_PATH)
print("IMAGES_DIR:", IMAGES_DIR)
print("BEST_MODEL_PATH:", BEST_MODEL_PATH)


CSV_PATH: ./train.csv
IMAGES_DIR: ./images
BEST_MODEL_PATH: ./modelos_guardados_isic2019\isic2019_mobilenetv2_best.keras


In [3]:
df = pd.read_csv(CSV_PATH)
df = df[df["benign_malignant"].isin(["benign", "malignant"])].copy()

df["image_stem"] = df["image_name"].astype(str).str.replace(r"\.(jpg|jpeg|png)$", "", regex=True)

df["filepath"] = df["image_stem"].apply(lambda s: os.path.join(IMAGES_DIR, f"{s}.jpg"))

df["label"] = (df["benign_malignant"] == "malignant").astype(int)

exists_mask = df["filepath"].apply(os.path.exists)
df = df[exists_mask].copy()

print("Total filas válidas:", len(df))
print(df[["benign_malignant","label"]].value_counts())


Total filas válidas: 25331
benign_malignant  label
benign            0        20809
malignant         1         4522
Name: count, dtype: int64


In [4]:
df_m = df[df["label"] == 1].sample(n=N_MALIGNANT, random_state=SEED)
df_b = df[df["label"] == 0].sample(n=N_BENIGN, random_state=SEED)
df_bal = pd.concat([df_m, df_b]).sample(frac=1.0, random_state=SEED).reset_index(drop=True)

print("Distribución en dataset:")
print("  Benignos:", (df_bal["label"]==0).sum(), f"({(df_bal['label']==0).mean()*100:.1f}%)")
print("  Malignos:", (df_bal["label"]==1).sum(), f"({(df_bal['label']==1).mean()*100:.1f}%)")


Distribución en dataset:
  Benignos: 4521 (50.0%)
  Malignos: 4521 (50.0%)


In [5]:
df_train, df_tmp = train_test_split(
    df_bal, test_size=(1.0-TRAIN_SIZE), random_state=SEED, stratify=df_bal["label"]
)

val_fraction_of_tmp = VAL_SIZE / (VAL_SIZE + TEST_SIZE)
df_val, df_test = train_test_split(
    df_tmp, test_size=(1.0-val_fraction_of_tmp), random_state=SEED, stratify=df_tmp["label"]
)

print("DIVISIÓN DE DATOS:")
print(f"  Train: {len(df_train)} muestras ({len(df_train)/len(df_bal)*100:.1f}%)")
print(f"  Validación: {len(df_val)} muestras ({len(df_val)/len(df_bal)*100:.1f}%)")
print(f"  Test: {len(df_test)} muestras ({len(df_test)/len(df_bal)*100:.1f}%)")
print(f"  Total: {len(df_bal)} muestras")

print("\nDistribución por clase (Train):")
print(df_train["label"].value_counts().rename({0:"Benignos",1:"Malignos"}))


DIVISIÓN DE DATOS:
  Train: 7233 muestras (80.0%)
  Validación: 904 muestras (10.0%)
  Test: 905 muestras (10.0%)
  Total: 9042 muestras

Distribución por clase (Train):
label
Malignos    3617
Benignos    3616
Name: count, dtype: int64


In [6]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.10,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_gen = train_datagen.flow_from_dataframe(
    df_train,
    x_col="filepath",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode="raw",
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED
)

val_gen = val_datagen.flow_from_dataframe(
    df_val,
    x_col="filepath",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode="raw",
    batch_size=BATCH_SIZE,
    shuffle=False
)

test_gen = val_datagen.flow_from_dataframe(
    df_test,
    x_col="filepath",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode="raw",
    batch_size=BATCH_SIZE,
    shuffle=False
)


Found 7233 validated image filenames.
Found 904 validated image filenames.
Found 905 validated image filenames.


In [7]:
base = MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base.trainable = False

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name="accuracy"),
        tf.keras.metrics.AUC(name="auc"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
    ],
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [8]:
callbacks = [
    ModelCheckpoint(BEST_MODEL_PATH, monitor="val_auc", mode="max", save_best_only=True, verbose=1),
    EarlyStopping(monitor="val_auc", mode="max", patience=8, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor="val_auc", mode="max", factor=0.5, patience=3, min_lr=1e-6, verbose=1),
]


In [9]:
history1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/20
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 917ms/step - accuracy: 0.6200 - auc: 0.6531 - loss: 0.6777 - precision: 0.6185 - recall: 0.6405
Epoch 1: val_auc improved from None to 0.80049, saving model to ./modelos_guardados_isic2019\isic2019_mobilenetv2_best.keras
227/227 ━━━━━━━━━━━━━━━━━━━━ 233s 1s/step - accuracy: 0.6617 - auc: 0.7254 - loss: 0.6217 - precision: 0.6610 - recall: 0.6641 - val_accuracy: 0.7201 - val_auc: 0.8005 - val_loss: 0.5403 - val_precision: 0.7196 - val_recall: 0.7212 - learning_rate: 0.0010
Epoch 2/20
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 520ms/step - accuracy: 0.6992 - auc: 0.7731 - loss: 0.5737 - precision: 0.6946 - recall: 0.7081
Epoch 2: val_auc improved from 0.80049 to 0.80664, saving model to ./modelos_guardados_isic2019\isic2019_mobilenetv2_best.keras
227/227 ━━━━━━━━━━━━━━━━━━━━ 126s 555ms/step - accuracy: 0.7048 - auc: 0.7837 - loss: 0.5616 - precision: 0.7040 - recall: 0.7069 - val_accuracy: 0.7301 - val_auc: 0.8066 - val_loss: 0.5388 - val_precision: 0.7723

In [10]:
base.trainable = True

for layer in base.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name="accuracy"),
        tf.keras.metrics.AUC(name="auc"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
    ],
)

history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/40
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 578ms/step - accuracy: 0.6753 - auc: 0.7349 - loss: 0.6592 - precision: 0.6976 - recall: 0.6301
Epoch 1: val_auc improved from 0.83952 to 0.83996, saving model to ./modelos_guardados_isic2019\isic2019_mobilenetv2_best.keras
227/227 ━━━━━━━━━━━━━━━━━━━━ 148s 622ms/step - accuracy: 0.6884 - auc: 0.7561 - loss: 0.6263 - precision: 0.6989 - recall: 0.6622 - val_accuracy: 0.7201 - val_auc: 0.8400 - val_loss: 0.5442 - val_precision: 0.8373 - val_recall: 0.5465 - learning_rate: 1.0000e-05
Epoch 2/40
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 570ms/step - accuracy: 0.7042 - auc: 0.7906 - loss: 0.5725 - precision: 0.7173 - recall: 0.6948
Epoch 2: val_auc did not improve from 0.83996
227/227 ━━━━━━━━━━━━━━━━━━━━ 137s 603ms/step - accuracy: 0.7091 - auc: 0.7906 - loss: 0.5724 - precision: 0.7109 - recall: 0.7050 - val_accuracy: 0.7035 - val_auc: 0.8370 - val_loss: 0.5764 - val_precision: 0.8459 - val_recall: 0.4978 - learning_rate: 1.0000e-05
Epoch 3/40
227/227 ━━━

In [11]:
best_model = tf.keras.models.load_model(BEST_MODEL_PATH)

y_true = df_test["label"].values
y_prob = best_model.predict(test_gen, verbose=0).ravel()
y_pred = (y_prob >= 0.5).astype(int)

test_auc = roc_auc_score(y_true, y_prob)
print("Test AUC:", round(test_auc, 4))
print("\nClassification report:")
print(classification_report(y_true, y_pred, digits=4))
print("Confusion matrix:\n", confusion_matrix(y_true, y_pred))


Test AUC: 0.8962

Classification report:
              precision    recall  f1-score   support

           0     0.8357    0.7859    0.8100       453
           1     0.7975    0.8451    0.8206       452

    accuracy                         0.8155       905
   macro avg     0.8166    0.8155    0.8153       905
weighted avg     0.8166    0.8155    0.8153       905

Confusion matrix:
 [[356  97]
 [ 70 382]]
